# Загрузка библиотек

In [ ]:
!pip install optuna -q
!pip install catboost -q
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import optuna
from optuna.integration import CatBoostPruningCallback

import seaborn as sns
import datetime
from wordcloud import WordCloud, STOPWORDS

import pickle

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


фонтанка, лента, риа, дзен?

# Фиксируем значения

In [ ]:
RND_STATE = 7575
N_EST = 1000
sampler = optuna.samplers.TPESampler(seed=RND_STATE)

# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Вероятно большая часть данных получена с сайта Лента.ру. Также видимо есть фонтанка. И возможно риа?

In [ ]:
class Paths:
    path_sub = '/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv'
    path_prep_test = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_04_01.csv'
    path_prep_train = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_04_01.csv'

paths = Paths()

# Navec

In [ ]:
# !pip install navec

In [ ]:
# !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2024-01-03 21:30:48--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  7.96MB/s    in 4.5s    

2024-01-03 21:30:54 (5.64 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



In [ ]:
# from navec import Navec

# path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
# navec = Navec.load(path)

In [ ]:
# HIDDEN = 300

# NewCols = ['col'+str(i) for i in range(HIDDEN)]

In [ ]:
# def get_mean_navec_vector(sentence):
#     Sum = 0
#     Count = 0

#     words = sentence.split()

#     for w in words:
#         if w in navec:
#             Sum += navec[w]
#             Count += 1

#     return Sum / Count

In [ ]:
# train = pd.read_csv(paths.path_prep_train)
# test = pd.read_csv(paths.path_prep_test)

In [ ]:
# train.shape

(39146, 7)

In [ ]:
# train_navec = train.copy()
# test_navec = test.copy()

# train_navec['vectors'] = train_navec['content_clean'].map(get_mean_navec_vector)
# test_navec['vectors'] = test_navec['content_clean'].map(get_mean_navec_vector)

In [ ]:
# train_navec[NewCols] = pd.DataFrame(train_navec['vectors'].tolist(), index=train_navec.index)
# test_navec[NewCols] = pd.DataFrame(test_navec['vectors'].tolist(), index=test_navec.index)

In [ ]:
# train_navec.drop(['vectors'], axis=1, inplace=True)
# test_navec.drop(['vectors'], axis=1, inplace=True)

In [ ]:
# train_navec.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_04_01.csv', index=False)
# test_navec.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_04_01.csv', index=False)

In [ ]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [ ]:
df_test = pd.read_csv(paths.path_test)

In [ ]:
df_test.iloc[23103].content

'Тренер баскетбольной команды средней школы в американском штате Нью-Джерси подвергся жестокому избиению со стороны своих подопечных. Видео опубликовано на TMZ Sports. Инцидент произошел 4 февраля. Полиция отреагировала на сообщение о том, что у дверей средней школы избивают одного из ее работников. На видео четверо молодых людей бьют мужчину руками и ногами. Как утверждает полиция, нападавшие — игроки школьной баскетбольной команды. Жертвой стал их тренер. Полиция Нью-Джерси не стала разглашать имена подозреваемых. Мэр города Ньюарк пообещал, что ученики школы понесут наказание.'

# Sentence Transformers

In [ ]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 906.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f5ee743c6d897d31cf03854a705310c22375554fdbc0e05ad7565f887e2acd69
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
query_embedding = model.encode('Пнекаофыжоажыф я тебя люблю?')
len(query_embedding)

768

In [ ]:
train_sentence_transformers = train.copy()
test_sentence_transformers = test.copy()

train_sentence_transformers['vectors'] = train_sentence_transformers['content_clean'].map(lambda x: model.encode(x))
test_sentence_transformers['vectors'] = test_sentence_transformers['content_clean'].map(lambda x: model.encode(x))

In [ ]:
train_sentence_transformers.head()

NameError: ignored

In [ ]:
HIDDEN = 768

NewCols = ['col_sent_trans'+str(i) for i in range(HIDDEN)]

In [ ]:
train_sentence_transformers[NewCols] = pd.DataFrame(train_sentence_transformers['vectors'].tolist(), index=train_sentence_transformers.index)
test_sentence_transformers[NewCols] = pd.DataFrame(test_sentence_transformers['vectors'].tolist(), index=test_sentence_transformers.index)

In [ ]:
train_sentence_transformers.drop(['vectors'], axis=1, inplace=True)
test_sentence_transformers.drop(['vectors'], axis=1, inplace=True)

In [ ]:
train_sentence_transformers.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_05_01.csv', index=False)
test_sentence_transformers.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_05_01.csv', index=False)

# Modeling

# Загрузка полностью подготовленных данных.

In [ ]:
train = pd.read_csv(paths.path_prep_train)
test = pd.read_csv(paths.path_prep_test)

In [ ]:
# Удаляем дубли
train.drop_duplicates(inplace=True)
# Удаляем пропуски
train.dropna(inplace=True)

print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}')

train shape = (39146, 307),  test shape = (26275, 306)
train, test is null: (False, False)


In [ ]:
text_features = ['content', 'content_clean']
# cat_features_indxs = [1]
targets = ['topic']
# cat_cols = ['content', 'content_clean']

filtered_features = [i for i in train.columns if (i not in targets)]

## Разделение на фичи и целевую переменную.

In [ ]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["topic"]

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

# CustomSelectColumns()

In [ ]:
class CustomSelectColumns():
    '''
    .. notes::
    Класс для вытаскивания из датасета только требуемых колонок
    '''
    def __init__(self,
                 fil_feats: list[str]=None,
                 drop: bool=False):
        self.fil_feats = fil_feats
        self.drop = drop

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        if self.drop == False:
            X_filtered = X_[self.fil_feats]
            X_filtered = pd.DataFrame(X_filtered)
            return pd.DataFrame(X_filtered)
        else:
            X_.drop(self.fil_feats, axis=1, inplace=True)
            return X_

## Базовая модель - catboost text features

In [ ]:
# X_train = X_train[: 1000]
# y_train = y_train[: 1000]

### pipeline variant

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content', 'content_clean'], drop=True)),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                          #  text_features=text_features
                                           ))
    ])

# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

# print(X.columns)

Catboost_pipe.fit(X, y,
                  # Catboost_simple__text_features=['content'],
                  Catboost_simple__verbose=10,
                  Catboost_simple__early_stopping_rounds=10,
                  # Catboost_simple__eval_set=(X_val[['content']], y_val)
                  )

NameError: ignored

In [ ]:
Catboost_pipe

NameError: ignored

In [ ]:
Catboost_pipe[-1].best_score_

NameError: ignored

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_10.csv', index=False)

In [ ]:
# saving for blending
df_sub_collect = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub_collect['Catboost_no_test'] = preds

In [ ]:
df_sub_collect.head()

,topic,index,Catboost_no_test
0,0,0,0
1,0,1,6
2,0,2,4
3,0,3,0
4,0,4,4


In [ ]:
with open(f'Catboost_pipe_model_new.pkl', "wb") as f:
        pickle.dump(Catboost_pipe, f)

### Optuna для catboost

In [ ]:
# def fit_catboost(trial, train, val):
#     X_train, y_train = train
#     X_val, y_val = val

#     X_eval = X_val.copy()

#     X_eval.drop(text_features, axis=1, inplace=True)

#     param = {
#         'iterations' : 500,
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.04, step=0.001),
#         "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 30, 70, step=10),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 0.4, step=0.1),
#         "auto_class_weights": trial.suggest_categorical("auto_class_weights",
#                                                         [#"SqrtBalanced",
#                                                          #"Balanced",
#                                                          "None"]),
#         "depth": trial.suggest_int("depth", 9, 10),
#         "boosting_type": trial.suggest_categorical("boosting_type", [#"Ordered",
#                                                                      "Plain"]),
#         "bootstrap_type": trial.suggest_categorical("bootstrap_type",
#                                                               [#"Bayesian",
#                                                                "Bernoulli",
#                                                                #"MVS"
#                                                                ]),
#         "used_ram_limit": "10gb",
#         "eval_metric": "Accuracy",
#         # 'loss_function': 'MultiClass'
#     }


#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)

#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)


#     clf = Pipeline([('col_selector', CustomSelectColumns(['content', 'content_clean'], drop=True)),
#                     ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
#                            iterations=200,
#                            thread_count=-1,
#                            random_seed=RND_STATE))])
#     #  Создаем объект callback
#     pruning_callback = CatBoostPruningCallback(trial, "Accuracy")


#     clf.fit(
#         X_train,
#         y_train,
#         Catboost_simple__eval_set=(X_eval, y_val),
#         Catboost_simple__verbose=50,
#         Catboost_simple__plot=False,
#         Catboost_simple__early_stopping_rounds=10,
#         # Catboost_simple__callbacks=[pruning_callback],
#     )
#     # print('Got here')
#     # запускаем процесс прунинга
#     # pruning_callback.check_pruned()

#     y_pred = clf.predict(X_val)
#     return clf, y_pred

In [ ]:
# kf = KFold(n_splits=3, shuffle=True, random_state=RND_STATE)

# for train_idx, valid_idx in kf.split(X_train):
#         train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
#         valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

#         # Подаем trials для перебора
#         model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше

In [ ]:
# def objective(trial, return_models=False):
#     n_splits = 3
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

#     scores_simple_acc, scores_balanced_acc, models = [], [], []

#     for train_idx, valid_idx in kf.split(X_train):
#         train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
#         valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

#         # Подаем trials для перебора
#         model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше

#         scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
#         # scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
#         models.append(model)
#         break

#     result = np.mean(scores_simple_acc)
#     # result = scores_balanced_acc
#     print(f'result = {result}')

#     if return_models:
#         return result, models
#     else:
#         return result

In [ ]:
# from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback
# from optuna.integration.pytorch_lightning import PyTorchLightningPruningCallback
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# # optuna.logging.set_verbosity(optuna.logging.ERROR)

In [ ]:
# study = optuna.create_study(direction="maximize",
#                             # pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
#                             study_name="Catboost_clf_no_txt",
#                             sampler=sampler)
# study.optimize(objective,
#                n_trials=40,
#                n_jobs=-1,
#                show_progress_bar=True)

[I 2024-01-03 23:04:14,867] A new study created in memory with name: Catboost_clf_no_txt


  0%|          | 0/40 [00:00<?, ?it/s]

Learning rate set to 0.212191
Learning rate set to 0.212191
0:	learn: 0.5612396	test: 0.5534055	best: 0.5534055 (0)	total: 5.42s	remaining: 17m 58s
0:	learn: 0.5612396	test: 0.5534055	best: 0.5534055 (0)	total: 5.31s	remaining: 17m 37s
100:	learn: 0.8645878	test: 0.8090813	best: 0.8100393 (98)	total: 4m 7s	remaining: 4m 2s
Stopped by overfitting detector  (5 iterations wait)

bestTest = 0.8100392758
bestIteration = 98

Shrink model to first 99 iterations.
result = nan
[W 2024-01-03 23:05:55,184] Trial 2 failed with parameters: {'learning_rate': 0.028000000000000004, 'l2_leaf_reg': 60, 'colsample_bylevel': 0.3, 'auto_class_weights': 'None', 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9343317298807932} because of the following error: The value nan is not acceptable.
[W 2024-01-03 23:05:55,187] Trial 2 failed with value nan.
result = nan
[W 2024-01-03 23:06:36,284] Trial 3 failed with parameters: {'learning_rate': 0.015, 'l2_leaf_reg': 50, 'colsampl

KeyboardInterrupt: ignored

In [ ]:
# print("Number of finished trials: {}".format(len(study.trials)))
# print("Best trial:")
# trial = study.best_trial
# print("  Value: {}".format(trial.value))
# print("  Params: ")

# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# valid_scores, models = objective(
#     optuna.trial.FixedTrial(study.best_params),
#     return_models=True,
# )

In [ ]:
# # История изменения от числа испытаний
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# params = ['learning_rate',
#           'l2_leaf_reg',
#           'colsample_bylevel',
#           'auto_class_weights',
#           'depth',
#           'bootstrap_type',
#           'boosting_type']

# optuna.visualization.plot_slice(study, params=params, target_name='accuracy')

## Базовая модель после optuna

### pipeline variant

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content_clean'], drop=True)),
    ('Catboost_simple', CatBoostClassifier(#depth=9,
                          #  l2_leaf_reg=50,
                          #  learning_rate=0.018,
                           colsample_bylevel=0.3,
                           auto_class_weights='None',
                          #  bootstrap_type= 'Bernoulli',
                          #  boosting_type="Plain",
                           eval_metric= 'Accuracy',
                           iterations=500,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=['content']))
    ])

X_eval = X_val.copy()

X_eval.drop('content_clean', axis=1, inplace=True)

Catboost_pipe.fit(X_train, y_train,
                  Catboost_simple__text_features=['content'],
                  Catboost_simple__verbose=50,
                  Catboost_simple__early_stopping_rounds=200,
                  Catboost_simple__eval_set=(X_eval, y_val)
                  )

Learning rate set to 0.150743
0:	learn: 0.7141397	test: 0.7217114	best: 0.7217114 (0)	total: 4.3s	remaining: 35m 48s


KeyboardInterrupt: ignored

In [ ]:
Catboost_pipe[-1].best_score_

{'learn': {'Accuracy': 0.9367096691786946, 'MultiClass': 0.18721112037821533},
 'validation': {'Accuracy': 0.9066411238825032,
  'MultiClass': 0.2766682148598085}}

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_11.csv', index=False)

In [ ]:
with open(f'Catboost_pipe_model_opt.pkl', "wb") as f:
        pickle.dump(Catboost_pipe, f)

NameError: ignored

## Simple models

In [ ]:
# Для получения вектора предложения используем tf_idf
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model.wv
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [ ]:
with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/model_w2v.pkl", "rb") as f:
        model_w2v = pickle.load(f)

## Логистическая регрессия

### W2V

In [ ]:
# pipeline variant
%%time
pipe_log = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                 ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.87      0.85      3374
           1       0.81      0.83      0.82       924
           2       0.85      0.79      0.82       568
           3       0.83      0.82      0.83       938
           4       0.97      0.96      0.97       835
           5       0.59      0.48      0.53        46
           6       0.93      0.95      0.94       193
           7       0.87      0.79      0.83       412
           8       0.87      0.80      0.83       540

    accuracy                           0.85      7830
   macro avg       0.84      0.81      0.82      7830
weighted avg       0.85      0.85      0.85      7830

CPU times: user 3min 40s, sys: 24.3 s, total: 4min 5s
Wall time: 3min 10s


In [ ]:
# simple variant
# %%time
# pipe_log = Pipeline([('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
#                  ('scaler', StandardScaler()),
#                  ('clf', LogisticRegression(random_state=RND_STATE,
#                                             max_iter=10000,
#                                             # class_weight='balanced'
#                                             ))])

# pipe_log.fit(X_train, y_train)

# print(classification_report(y_val, pipe_log.predict(X_val)))

In [ ]:
# precision    recall  f1-score   support

#          0.0       0.78      0.82      0.80      4287
#          1.0       0.77      0.77      0.77      2110
#          2.0       0.77      0.68      0.72       981
#          3.0       0.85      0.81      0.83      1264
#          4.0       0.97      0.97      0.97      1321
#          5.0       0.60      0.47      0.53        87
#          6.0       0.95      0.96      0.95       203
#          7.0       0.87      0.83      0.85       612
#          8.0       0.80      0.78      0.79       935

#     accuracy                           0.81     11800
#    macro avg       0.82      0.79      0.80     11800
# weighted avg       0.81      0.81      0.81     11800

# CPU times: user 7min 54s, sys: 49.5 s, total: 8min 43s

### Optuna для лог_рег

In [ ]:
def fit_log_reg(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"dual": False,
             "C": trial.suggest_float("C", 2.5, 3, step=0.1),
             "solver": trial.suggest_categorical("solver",
                                                 ["sag",
                                                  #"saga",
                                                  "lbfgs"
                                                  ]),
             "random_state": RND_STATE,
             "max_iter": trial.suggest_int("max_iter", 5500, 6300, step=200),
             'multi_class': trial.suggest_categorical("multi_class",
                                                      [#'auto',
                                                       'ovr',
                                                       #'multinomial'
                                                       ]),
             'n_jobs': -1,
             'verbose': 0
    }

    # print(f'param["solver"] = {param["solver"]}')
    if param["solver"] == "saga":
        param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l1',
                                              'l2', #'elasticnet',
                                              None])
    else:
         param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l2',
                                              #None
                                              ])

    clf = LogisticRegression(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_log(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_log_reg(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)
    # print(f'result = {result}')

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="Log_Reg_clf")
study.optimize(objective_log,
               n_trials=10,
               n_jobs=-1,
               show_progress_bar=True)

[I 2024-01-02 15:41:48,183] A new study created in memory with name: Log_Reg_clf


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-01-02 16:04:47,776] Trial 0 failed with parameters: {'C': 2.3, 'solver': 'lbfgs', 'max_iter': 9010, 'multi_class': 'auto', 'penalty': None} because of the following error: KeyboardInterrupt().
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in __call__
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 589, in <listcomp>
    return [func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 123, in __call__
    return self.function(*args, **kwargs)
  File "/

KeyboardInterrupt: ignored

### new begginings

In [ ]:
from mlxtend.feature_selection import ColumnSelector

In [ ]:
class CustomColumns():
    '''
    .. notes::
    Класс для вытаскивания из датасета только требуемых колонок
    '''
    def __init__(self,
                 fil_feats: list[str]=None, clean: bool=False):
        # self.fil_feats = fil_feats
        self.clean = clean

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        X_filtered = X_[self.fil_feats]
        X_filtered = pd.DataFrame(X_filtered)
        if self.clean:
            return X_filtered[self.fil_feats[0]].str.split()
        else:
            return pd.DataFrame(X_filtered)

Index(['content', 'content_clean', 'mean_w2v', 'max_w2v', 'mean_tf', 'max_tf'], dtype='object')

In [ ]:
%%time
pipe_log = Pipeline([('col_selector', ColumnSelector(cols=['content_clean'],drop_axis=True)),
                     ('tfidf', TfidfVectorizer()),
                     ('scaler', StandardScaler(with_mean=False)),
                     ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))
                 ])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      3374
           1       0.82      0.84      0.83       924
           2       0.84      0.80      0.82       568
           3       0.87      0.83      0.85       938
           4       0.97      0.96      0.97       835
           5       0.84      0.35      0.49        46
           6       0.95      0.91      0.93       193
           7       0.90      0.76      0.83       412
           8       0.89      0.75      0.81       540

    accuracy                           0.86      7830
   macro avg       0.88      0.79      0.82      7830
weighted avg       0.86      0.86      0.86      7830

CPU times: user 1min 48s, sys: 1min 7s, total: 2min 55s
Wall time: 1min 53s


In [ ]:
%%time
pipe_log = Pipeline([('col_selector', ColumnSelector(cols=['mean_w2v', 'max_w2v', 'mean_tf', 'max_tf'],drop_axis=True)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))
                 ])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

              precision    recall  f1-score   support

           0       0.48      0.89      0.62      3374
           1       0.49      0.32      0.39       924
           2       0.00      0.00      0.00       568
           3       0.32      0.03      0.05       938
           4       0.41      0.40      0.40       835
           5       0.00      0.00      0.00        46
           6       0.00      0.00      0.00       193
           7       0.00      0.00      0.00       412
           8       0.20      0.01      0.02       540

    accuracy                           0.47      7830
   macro avg       0.21      0.18      0.16      7830
weighted avg       0.36      0.47      0.36      7830

CPU times: user 2.1 s, sys: 1.23 s, total: 3.33 s
Wall time: 2.9 s


## SVM

### W2V

In [ ]:
#pipeline version
%%time
pipe_svc = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SVC(random_state=RND_STATE,
                             max_iter=10000,
                            #  class_weight='balanced'
                             ))])

pipe_svc.fit(X_train, y_train)

print(classification_report(y_val, pipe_svc.predict(X_val)))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3374
           1       0.83      0.87      0.85       924
           2       0.86      0.81      0.84       568
           3       0.86      0.86      0.86       938
           4       0.97      0.98      0.97       835
           5       0.76      0.35      0.48        46
           6       0.96      0.92      0.94       193
           7       0.90      0.86      0.88       412
           8       0.90      0.82      0.86       540

    accuracy                           0.88      7830
   macro avg       0.88      0.82      0.84      7830
weighted avg       0.88      0.88      0.88      7830

CPU times: user 4min 4s, sys: 1.31 s, total: 4min 5s
Wall time: 4min 8s


### Optuna для svm

In [ ]:
def fit_svm(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"C": trial.suggest_float("C", 1, 1.5, step=0.5),
             "kernel": trial.suggest_categorical("kernel",
                                                 [#"linear",
                                                  "poly",
                                                  #"rbf",
                                                  #'sigmoid'
                                                  ]),
             "random_state": RND_STATE,
             "degree": trial.suggest_int("degree", 3, 4, step=1),
             "max_iter": trial.suggest_int("max_iter", 5000, 6000, step=1000),
             'decision_function_shape': trial.suggest_categorical("decision_function_shape",
                                                      ['ovo',
                                                      #  'ovr'
                                                       ]),
             'verbose': 0
    }

    clf = SVC(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_svm(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_svm(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="SVM_clf")
study.optimize(objective_svm,
               n_trials=4,
               n_jobs=-1,
               show_progress_bar=True)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# # История изменения от числа испытаний
# optuna.visualization.plot_optimization_history(study)

### new beginnings

In [ ]:
# %%time
# pipe_log = Pipeline([('col_selector', ColumnSelector(cols=['content_clean'],drop_axis=True)),
#                      ('tfidf', TfidfVectorizer()),
#                      ('scaler', StandardScaler(with_mean=False)),
#                      ('clf', SVC(random_state=RND_STATE,
#                                             max_iter=10000,
#                                             # class_weight='balanced'
#                                             ))
#                  ])

# pipe_log.fit(X_train, y_train)

# print(classification_report(y_val, pipe_log.predict(X_val)))

# Новый раздел

# Stack

In [ ]:
# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content'])),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34df6a0>),
                                                ('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x7e2af34dfc70>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2ae6042770>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer o...
                                                     random_state=7575))])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34ded70>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer object at 0x7e2af34df610>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(max_iter=10000,
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
%%time
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34df6a0>),
                                                ('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x7e2af34dfc70>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2ae6042770>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer o...
                                                     random_state=7575))])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34ded70>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer object at 0x7e2af34df610>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(max_iter=10000,
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
preds_st = stacking_classifier.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)

0.8594459859876837

In [ ]:
# 0.854229941129818 - catboost + log_reg

In [ ]:
accuracy_score(y_true=y_val, y_pred=preds_st)

0.90242656449553

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_5.csv', index=False)

# На всём датасете

In [ ]:
%%time
stacking_classifier.fit(X, y)

CPU times: user 1min 10s, sys: 16.8 s, total: 1min 27s
Wall time: 1h 38min 51s


StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34df6a0>),
                                                ('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x7e2af34dfc70>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2ae6042770>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer o...
                                                     random_state=7575))])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7e2af34ded70>),
                                                ('w2v_tfidf',
                                                 <__main__.TfidfEmbeddingVectorizer object at 0x7e2af34df610>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(max_iter=10000,
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
preds_st = stacking_classifier.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)

0.9002567925976034

In [ ]:
accuracy_score(y_true=y_val, y_pred=preds_st)

0.9173690932311622

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_7.csv', index=False)

# TAbnetCl

In [ ]:
!pip install pytorch-tabnet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.0 MB/s eta 0:00:00


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
%%time
TabNet_pipe = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', TabNetClassifier(n_steps=5))])

TabNet_pipe.fit(X_train, y_train,
                #  clf__eval_set=(X_val, y_val)
                 )

print(classification_report(y_val, TabNet_pipe.predict(X_val)))

TypeError: ignored

In [ ]:
TabNetClassifier( )

# Another embedding

In [ ]:
!pip install navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2024-01-03 15:44:53--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.0MB/s    in 4.3s    

2024-01-03 15:44:58 (11.9 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
from navec import Navec

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
from slovnet.model.emb import NavecEmbedding

In [ ]:
# pipeline variant
%%time
pipe_log = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                 ('w2v_tfidf', TfidfEmbeddingVectorizer(modelw2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

NameError: ignored

In [ ]:
def get_Mean_w2v_vector(sentence):
    Sum = 0
    Count = 0

    words = sentence.split()

    # print(words)

    for w in words:
        # print(w in model_w2v.wv)
        if w in model_w2v.wv:
            Sum += model_w2v.wv[w]
            Count += 1
    # print(Count)
    return np.mean(Sum / Count)

In [ ]:
# NewCols = ['col'+str(i) for i in range(HIDDEN)]


# trainw2vn = trainw2v.copy()
# test = test.copy()

trainw2v['mean_w2v'] = trainw2v['content_clean'].map(get_Mean_w2v_vector)
# test['mean_w2v'] = test['content_clean'].map(get_Mean_w2v_vector)

In [ ]:
test.to_csv('test.csv', index=False)

In [ ]:
trainw2v.head()

,content,topic,content_clean,max_w2v,mean_w2v
0,"Поклонники Гарри Поттера со всего мира смотрят спецэпизод, приуроченный к 20-летию самой волшебной франшизы. 1 январ...",0,поклонник гарри поттер мир смотреть спецэпизод приурочить волшебный франшиза январь документальный фильм секрет созд...,0.881894,-0.015332
1,Египетский курорт засыпало градом 1 января. В некоторых районах он был крупным и причинял прохожим боль. Фото и виде...,0,египетский курорт засыпало град январь некоторый район крупный причинять прохожий боль фото видео курорт опубликоват...,0.958959,-0.004325
2,Главное управление по Краснодарскому краю МЧС России 1 января выпустило экстренное предупреждение по лавиноопасности...,0,главный управление краснодарский край мчс россия январь выпустить экстренный предупреждение лавиноопасность информац...,0.740297,0.004588
3,В новогоднюю ночь в родильных домах Петербурга и акушерских отделениях городских больниц на свет появились 34 младен...,0,новогодний ночь родильный дом петербург акушерский отделение городской больница свет появиться младенец сообщить смо...,1.164321,-0.024724
4,"Договор с названием RCEP (Regional Comprehensive Economic Partnership, или Всестороннее региональное экономическое п...",0,договор название rcep regional comprehensive economic partnership всесторонний региональный экономический партнерств...,1.432906,-0.011452


In [ ]:
t = TfidfEmbeddingVectorizer(model_w2v).fit(X, y)

In [ ]:
t.transform(X['content_clean']).mean()

0.0

In [ ]:
pipe_tr = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                    ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v))])

In [ ]:
f = trainw2v.copy()

In [ ]:
# trainw2v.shape

In [ ]:
r = pipe_tr.fit_transform(trainw2v[['content_clean']])

In [ ]:
meantf = [np.mean(el) for el in r]

In [ ]:
# trainw2v['mean_tf'] = meantf

In [ ]:
# test.head()

In [ ]:
# trainw2v = trainw2v[['content', 'topic',	'content_clean', 'mean_w2v', 'max_w2v', 'mean_tf',	'max_tf']]

In [ ]:
# test.to_csv('test.csv', index=False)

# new begginings

In [ ]:
X = trainw2v.drop(targets, axis=1, errors="ignore")
y = trainw2v["topic"]

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

In [ ]:
# X.head()

In [ ]:
clf_new = CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=['content', 'content_clean'])


train_dataset = Pool(data=X_train, label=y_train, text_features=['content', 'content_clean'])
eval_dataset = Pool(data=X_val, label=y_val, text_features=['content', 'content_clean'])

clf_new.fit(X_train, y_train,
                  text_features=['content', 'content_clean'],
                  verbose=5,
                  early_stopping_rounds=10,
                  eval_set=(X_val, y_val)
                  )

Learning rate set to 0.213919
0:	learn: 0.8023694	test: 0.8148148	best: 0.8148148 (0)	total: 7.77s	remaining: 25m 46s
5:	learn: 0.8358986	test: 0.8469987	best: 0.8473819 (4)	total: 40.1s	remaining: 21m 36s
10:	learn: 0.8519926	test: 0.8630907	best: 0.8630907 (10)	total: 1m 11s	remaining: 20m 33s
15:	learn: 0.8594648	test: 0.8674330	best: 0.8674330 (13)	total: 1m 41s	remaining: 19m 26s
20:	learn: 0.8675437	test: 0.8743295	best: 0.8743295 (20)	total: 2m 11s	remaining: 18m 39s
25:	learn: 0.8709925	test: 0.8768838	best: 0.8771392 (23)	total: 2m 41s	remaining: 18m 3s
30:	learn: 0.8751756	test: 0.8796935	best: 0.8796935 (30)	total: 3m 12s	remaining: 17m 29s
35:	learn: 0.8768681	test: 0.8804598	best: 0.8804598 (35)	total: 3m 45s	remaining: 17m 5s
40:	learn: 0.8806680	test: 0.8853129	best: 0.8853129 (40)	total: 4m 14s	remaining: 16m 27s
45:	learn: 0.8831268	test: 0.8863346	best: 0.8868455 (44)	total: 4m 43s	remaining: 15m 49s
50:	learn: 0.8859050	test: 0.8882503	best: 0.8882503 (50)	total: 5m 

In [ ]:
clf_new.best_score_

{'learn': {'Accuracy': 0.9034359432877762, 'MultiClass': 0.2878800430623185},
 'validation': {'Accuracy': 0.8978288633461047,
  'MultiClass': 0.3063018108320806}}

In [ ]:
# trainw2vn['vectors_tf_idf'] = trainw2vn['content_clean'].map(TfidfEmbeddingVectorizer(model_w2v))

# Ru-bert

In [ ]:
# from deeppavlov.core.common.file import read_json
# from deeppavlov import build_model, configs
# from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
# # ссылка для скачивания моделей http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html

In [ ]:
# class RU_BERT_CLASS:
#     def __init__(self, name):
#         bert_config = read_json(configs.embedder.bert_embedder)
#         bert_config['metadata']['variables']['BERT_PATH'] = os.path.join('./.', name)
#         self.m = build_model(bert_config)

#     def vectorizer(self, sentences):
#         return [sentence.split() for sentence in sentences]

#     def predict(self, tokens):
#         _, _, _, _, sent_max_embs, sent_mean_embs, _ = self.m(tokens)
#         return sent_mean_embs

# bert = RU_BERT_CLASS('rubert_cased_L-12_H-768_A-12_pt')
# get_similarity_values = similarity_values_wrapper(bert.predict, bert.vectorizer, distance_function=cosine_distances)
# evaluate(get_similarity_values, 'rubert')

# Navec

In [ ]:
!pip install navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2024-01-03 20:46:38--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  9.43MB/s    in 2.7s    

2024-01-03 20:46:41 (9.43 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



In [ ]:
from navec import Navec

path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
# len(navec['навек'])

300

In [ ]:
# 'e' in navec

False

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

In [ ]:
NewCols = ['col'+str(i) for i in range(300)]

In [ ]:
def get_mean_navec_vector(sentence):
    Sum = 0
    Count = 0

    words = sentence.split()

    for w in words:
        if w in navec:
            Sum += navec[w]
            Count += 1

    return Sum / Count

In [ ]:
X_train_navec = X_train.copy()
X_test_navec = X_val.copy()

X_train_navec['vectors'] = X_train['content_clean'].map(get_mean_navec_vector)
X_test_navec['vectors'] = X_val['content_clean'].map(get_mean_navec_vector)

In [ ]:
X_train_navec.head()

,content,content_clean,mean_w2v,max_w2v,mean_tf,max_tf,vectors
26460,Кадр: @reecejames Николай Костюков Английский ...,кадр reecejames николай костюков защитник челс...,-0.012824,1.339583,-0.071109,6.597387,"[0.040373437, -0.044753682, -0.014275469, 0.20..."
36062,Фото: Министерство обороны РФ / РИА Новости Ни...,фото министерство оборона риа новость никита а...,-0.041981,1.574258,-0.157066,6.142687,"[-0.04654625, -0.18393707, -0.037581947, 0.068..."
22197,Фото: Михаил Джапаридзе / ТАСС Алекса Аксакова...,фото михаил джапаридзе тасс алекс аксаков милл...,-0.020011,1.259819,-0.075489,5.312569,"[-0.0076509984, -0.14936692, -0.09365584, 0.17..."
29823,Фото: Pixabay Виктория Кондратьева В городе Ка...,фото pixabay виктория кондратьев город качкана...,-0.019750,1.335371,-0.096587,6.305456,"[0.020762736, -0.12128142, -0.08194063, 0.0959..."
16975,"В Башкирии чиновник обматерил местного жителя,...",башкирия чиновник обматерить местный житель пр...,0.004188,0.877347,0.024898,3.505953,"[0.026577257, -0.05295464, -0.006716688, 0.082..."


In [ ]:
X_train_navec[NewCols] = pd.DataFrame(X_train_navec['vectors'].tolist(), index= X_train_navec.index)
X_test_navec[NewCols] = pd.DataFrame(X_test_navec['vectors'].tolist(), index= X_test_navec.index)

In [ ]:
X_train_navec.head()

,content,content_clean,mean_w2v,max_w2v,mean_tf,max_tf,vectors,col0,col1,col2,...,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299
26460,Кадр: @reecejames Николай Костюков Английский ...,кадр reecejames николай костюков защитник челс...,-0.012824,1.339583,-0.071109,6.597387,"[0.040373437, -0.044753682, -0.014275469, 0.20...",0.040373,-0.044754,-0.014275,...,0.151177,0.267800,-0.014835,0.140306,-0.041970,0.003104,0.297582,0.113405,0.005165,-0.086479
36062,Фото: Министерство обороны РФ / РИА Новости Ни...,фото министерство оборона риа новость никита а...,-0.041981,1.574258,-0.157066,6.142687,"[-0.04654625, -0.18393707, -0.037581947, 0.068...",-0.046546,-0.183937,-0.037582,...,0.122373,0.112150,-0.086288,0.102340,-0.027085,0.096666,0.224928,-0.068760,0.072320,-0.132715
22197,Фото: Михаил Джапаридзе / ТАСС Алекса Аксакова...,фото михаил джапаридзе тасс алекс аксаков милл...,-0.020011,1.259819,-0.075489,5.312569,"[-0.0076509984, -0.14936692, -0.09365584, 0.17...",-0.007651,-0.149367,-0.093656,...,0.113310,0.149989,-0.116707,0.104497,-0.034415,0.059256,0.196581,0.031802,0.035446,-0.135028
29823,Фото: Pixabay Виктория Кондратьева В городе Ка...,фото pixabay виктория кондратьев город качкана...,-0.019750,1.335371,-0.096587,6.305456,"[0.020762736, -0.12128142, -0.08194063, 0.0959...",0.020763,-0.121281,-0.081941,...,0.155983,0.138679,0.056608,0.069123,-0.109356,0.129159,0.173831,0.103273,-0.001785,-0.071128
16975,"В Башкирии чиновник обматерил местного жителя,...",башкирия чиновник обматерить местный житель пр...,0.004188,0.877347,0.024898,3.505953,"[0.026577257, -0.05295464, -0.006716688, 0.082...",0.026577,-0.052955,-0.006717,...,0.167372,0.155685,-0.008896,0.100504,-0.033962,0.041272,0.088022,0.068250,0.039063,-0.126219


In [ ]:
X_train_navec.drop(['content','content_clean','vectors'], axis=1, inplace=True)
X_test_navec.drop(['content','content_clean', 'vectors'], axis=1, inplace=True)

In [ ]:
%%time
pipe_log = Pipeline([('scaler', StandardScaler()),
                     ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train_navec, y_train)

print(classification_report(y_val, pipe_log.predict(X_test_navec)))

              precision    recall  f1-score   support

           0       0.82      0.85      0.83      3374
           1       0.79      0.81      0.80       924
           2       0.82      0.74      0.78       568
           3       0.82      0.82      0.82       938
           4       0.97      0.97      0.97       835
           5       0.51      0.43      0.47        46
           6       0.92      0.95      0.94       193
           7       0.82      0.77      0.80       412
           8       0.84      0.77      0.81       540

    accuracy                           0.84      7830
   macro avg       0.81      0.79      0.80      7830
weighted avg       0.84      0.84      0.83      7830

CPU times: user 2min 14s, sys: 26.9 s, total: 2min 41s
Wall time: 1min 45s


In [ ]:
%%time
pipe_log = Pipeline([('scaler', StandardScaler()),
                     ('clf', XGBClassifier(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train_navec, y_train)

print(classification_report(y_val, pipe_log.predict(X_test_navec)))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84      3374
           1       0.79      0.83      0.81       924
           2       0.82      0.73      0.77       568
           3       0.83      0.79      0.81       938
           4       0.97      0.96      0.97       835
           5       0.92      0.24      0.38        46
           6       0.94      0.90      0.92       193
           7       0.87      0.76      0.81       412
           8       0.88      0.78      0.83       540

    accuracy                           0.84      7830
   macro avg       0.87      0.76      0.79      7830
weighted avg       0.84      0.84      0.84      7830

CPU times: user 4min 24s, sys: 994 ms, total: 4min 25s
Wall time: 2min 42s


# Transformers

In [ ]:
import torch
import transformers as ppb

In [ ]:
!pip install transformers -q

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# trainw2v.head()

In [ ]:
tokenized = trainw2v['content_clean'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (936 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy() # векторы текстов длины 768

In [ ]:
labels = trainw2v['topic']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [ ]:
accuracy_score(train_labels, lr_clf.predict(train_features)), accuracy_score(test_labels, lr_clf.predict(test_features))